In [1]:
%pip install xlrd

Note: you may need to restart the kernel to use updated packages.


## Data Preparation and Merging (Top 15 Countries Analysis)

In this section, we prepare and clean the datasets required for our analysis.  
We work with **three different datasets**: 
1. `Energy Indicators` dataset  
2. `World Bank GDP` dataset  
3. `ScimEn` (scientific ranking) dataset  

The goal is to create a unified dataset that combines these three sources and restricts it to the **Top 15 countries** by scientific rank.

---


## 1. Importing Required Libraries

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)


## 2. Loading Datasets

In [4]:
energy = pd.read_excel('Energy Indicators.xls', skiprows=17, skipfooter=38, usecols='C:F')
energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

GDP = pd.read_csv('world_bank.csv', skiprows=4)

ScimEn = pd.read_excel('scimagojr-3.xlsx')


## 3. Cleaning the Energy Data

In [5]:
# Convert Energy Supply from petajoules to gigajoules
energy['Energy Supply'] = energy['Energy Supply'] * 1_000_000

# Replace missing values
energy.replace('...', np.nan, inplace=True)

# Clean country names
energy['Country'] = energy['Country'].str.replace(r"\(.*\)", "", regex=True) \
                                     .str.replace(r"\d+", "", regex=True) \
                                     .str.strip()

# Manual replacements
country_replacements = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}
energy.replace({"Country": country_replacements}, inplace=True)

energy.sample(10)


/var/folders/xz/w2xjxjwd0n9bhv2r41r2rs4c0000gn/T/ipykernel_18068/2131334159.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy.replace('...', np.nan, inplace=True)


,Country,Energy Supply,Energy Supply per Capita,% Renewable
154,Palau,3000000,152.0,18.750000
145,New Zealand,876000000,196.0,72.873130
136,Montserrat,1000000,153.0,0.000000
206,Tunisia,430000000,39.0,3.492547
167,Romania,1339000000,68.0,34.384500
199,Tajikistan,106000000,13.0,99.742920
144,New Caledonia,51000000,200.0,20.430970
60,Dominica,2000000,28.0,36.633660
125,Maldives,15000000,43.0,0.000000
32,Burkina Faso,164000000,10.0,14.500680


## 4. Cleaning the GDP Data 

In [6]:
# Rename country column to match 'Energy' dataset
GDP.rename(columns={'Country Name': 'Country'}, inplace=True)

# Keep only years 2006–2015 (to align with Sciamago data timeframe)
years_to_keep = list(map(str, range(2006, 2016)))
GDP = GDP[['Country'] + years_to_keep]

# Renaming specific countries
GDP.replace({
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}, inplace=True)


# Preview
print(GDP.shape)
GDP.sample(10)


(264, 11)


,Country,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
48,Curacao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,El Salvador,2.074243e+10,2.153889e+10,2.181334e+10,2.112992e+10,2.141830e+10,2.189311e+10,2.230491e+10,2.271694e+10,2.304072e+10,2.360638e+10
50,Cyprus,2.337425e+10,2.452756e+10,2.542347e+10,2.490764e+10,2.524742e+10,2.534952e+10,2.472895e+10,2.326028e+10,2.267946e+10,2.303920e+10
47,Cuba,5.545431e+10,5.948147e+10,6.193022e+10,6.282767e+10,6.432820e+10,6.613130e+10,6.812475e+10,6.995461e+10,NaN,NaN
93,Hong Kong,2.019164e+11,2.149698e+11,2.195440e+11,2.141452e+11,2.286377e+11,2.396459e+11,2.437206e+11,2.512081e+11,2.577757e+11,2.638606e+11
168,North America,1.636271e+13,1.665824e+13,1.663040e+13,1.616585e+13,1.658352e+13,1.687366e+13,1.724058e+13,1.750951e+13,1.793538e+13,1.834658e+13
133,Least developed countries: UN classification,5.246739e+11,5.738811e+11,6.164057e+11,6.438681e+11,6.783207e+11,7.037128e+11,7.332169e+11,7.739316e+11,8.164271e+11,8.526801e+11
207,Sierra Leone,2.083567e+09,2.251468e+09,2.373040e+09,2.483374e+09,2.616611e+09,2.780637e+09,3.199014e+09,3.854411e+09,4.031360e+09,3.212944e+09
68,Estonia,2.189339e+10,2.358974e+10,2.231130e+10,1.902609e+10,1.949466e+10,2.097297e+10,2.205991e+10,2.240576e+10,2.305699e+10,2.330321e+10
35,Switzerland,5.415550e+11,5.639773e+11,5.768204e+11,5.645363e+11,5.812117e+11,5.917000e+11,5.983544e+11,6.089383e+11,6.204440e+11,6.260878e+11


## 5. Merging Datasets

In [11]:
# 1. Filter ScimEn to top 15 first
ScimEn_top15 = ScimEn[ScimEn['Rank'] <= 15]

# 2. Restrict GDP to just the years of interest
GDP = GDP[['Country'] + years_to_keep]

# 3. Merge only the needed data
df_top15 = pd.merge(ScimEn_top15, energy, how='inner', on='Country')
df_top15 = pd.merge(df_top15, GDP, how='inner', on='Country')

# 4. Set index to Country
df_top15 = df_top15.set_index('Country')

# 5. Reorder columns (optional, for grading consistency)
df_top15 = df_top15[[
    'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
    'Citations per document', 'H index', 'Energy Supply',
    'Energy Supply per Capita', '% Renewable'
] + years_to_keep]

# 6. Sort by rank
df_top15.sort_values(by='Rank', inplace=True)


In [23]:
df_top15.columns

Index(['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'],
      dtype='object')

In [25]:
# Checking whether the changes have been made to our dataset

def sanity_check(df_top15):
    print("Shape check:",df_top15.shape, " (Expected: (15, 20))")
    print("-" * 50)

    # Check index (countries)
    print("Number of countries:", len(df_top15.index), " (Expected: 15)")
    print("Countries:", list(df_top15.index))
    print("-" * 50)

    # Check Rank sorting
    print("Rank values:",df_top15['Rank'].tolist())
    print("-" * 50)

    # Spot check numeric values
    print("Energy & Renewable columns (first 5 rows):")
    print(df[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].head())
    print("-" * 50)

    print("GDP columns 2006–2015 (first 5 rows):")
    print(df.loc[:, '2006':'2015'].head())
    print("-" * 50)

# Run the check
sanity_check(df_top15)


Shape check: (15, 20)  (Expected: (15, 20))
--------------------------------------------------
Number of countries: 15  (Expected: 15)
Countries: ['China', 'United States', 'Japan', 'United Kingdom', 'Russian Federation', 'Canada', 'Germany', 'India', 'France', 'South Korea', 'Italy', 'Spain', 'Iran', 'Australia', 'Brazil']
--------------------------------------------------
Rank values: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
--------------------------------------------------
Energy & Renewable columns (first 5 rows):
  Energy Supply  Energy Supply per Capita  % Renewable
0  127191000000                      93.0     19.75491
1   90838000000                     286.0     11.57098
2   18984000000                     149.0     10.23282
3    7920000000                     124.0     10.60047
4   30709000000                     214.0     17.28868
--------------------------------------------------
GDP columns 2006–2015 (first 5 rows):
           2006          2007          2008  

### The following is a summary of what I worked on on this notebook:

## Step 1: Data Cleaning
- The `Energy` dataset was first cleaned by:  
  - Renaming countries to ensure consistency (e.g., removing footnotes and standardizing names).  
  - Converting "Energy Supply" to gigajoules and ensuring non-negative values.  
  - Keeping only relevant columns: `['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`.

- The `GDP` dataset was also cleaned by:  
  - Renaming country names where necessary.  
  - Keeping GDP data from **2006–2015**.  
  - Ensuring missing values are preserved (rather than dropped), since they may indicate unavailable data.

- The `ScimEn` dataset already contained a clean scientific ranking of countries.  
  - Did not need extensive cleaning here.

---

## Step 2: Merging Datasets
- I performed **two successive merges**:
  1. `ScimEn` with `Energy` on `"Country"`.
  2. The resulting dataframe with `GDP` on `"Country"`.

- The merge type was **inner join** to keep only countries appearing in all three datasets.  

**Intuition:**  
This ensures that the resulting dataset contains consistent records across all three sources, preventing mismatches or missing countries.

---

## Step 3: Restricting to Top 15 Countries
- From the merged dataset, I restricted the records to **only the top 15 countries by scientific rank (`Rank <= 15`)**.  
- This matches the problem requirement of focusing analysis on the highest-ranked scientific nations.

**Intuition:**  
By limiting to the top 15, I ensured that all subsequent analysis is relevant to the "elite group" of countries, avoiding skewness from including lower-ranked ones.

---

## Step 4: Selecting Relevant Columns
I explicitly kept only the columns needed for further analysis:  

- From **ScimEn**: `Rank`, `Documents`, `Citable documents`, `Citations`, `Self-citations`, `Citations per document`, `H index`.  
- From **Energy**: `Energy Supply`, `Energy Supply per Capita`, `% Renewable`.  
- From **GDP**: `2006` through `2015`.


---

## Step 5: Sorting by Rank
- Finally, the dataset was sorted by `"Rank"` in ascending order to maintain consistency in presentation.

---

### Final Result
The final dataset `df_top15` is a **clean, merged, and sorted dataframe** containing the top 15 ranked countries with:
- Scientific metrics (from `ScimEn`)  
- Energy indicators (from `Energy`)  
- GDP values (from `World Bank`, years 2006–2015)  

This dataset is now ready for further analysis, such as computing statistics, identifying trends, and exploring correlations.

